# The Price is Right

Today we build a more complex solution for estimating prices of goods.

1. Day 2.0 notebook: create a RAG database with our 400,000 training data
2. Day 2.1 notebook: visualize in 2D
3. Day 2.2 notebook: visualize in 3D
4. Day 2.3 notebook: build and test a RAG pipeline with GPT-4o-mini
5. Day 2.4 notebook: (a) bring back our Random Forest pricer (b) Create a Ensemble pricer that allows contributions from all the pricers

Phew! That's a lot to get through in one day!

## PLEASE NOTE:

We already have a very powerful product estimator with our proprietary, fine-tuned LLM. Most people would be very satisfied with that! The main reason we're adding these extra steps is to deepen your expertise with RAG and with Agentic workflows.

## We will go fast today! Hold on to your hat..

In [7]:
import sys
import os
# Add parent folder of current working directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
# Import module from parent folder

In [8]:
from LLMHandler import LLMHandler

In [1]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from testing import Tester

In [2]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [3]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
# Another import after Logging in to Hugging Face - thank you Trung N.!

from items import Item

In [5]:
openai = OpenAI()

In [13]:
# Load in the test pickle file
# See the section "Back to the PKL files" in the day2.0 notebook
# for instructions on obtaining this test.pkl file

with open('../week6/test.pkl', 'rb') as file:
    test = pickle.load(file)

In [14]:
def make_context(similars, prices):
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [58]:
system_message = "You estimate prices of items. Reply only with the price, no explanation\n\n eg. Price is $____"
handler = LLMHandler(system_message=system_message)
handler.TEXT_MODEL ="gemini-2.0-flash"
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return user_prompt

In [ ]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [17]:
DB = "products_vectorstore"

In [18]:
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [19]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [20]:
description(test[0])

"OEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K"

In [21]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [22]:
def vector(item):
    return model.encode([description(item)])

In [23]:
def find_similars(item):
    results = collection.query(query_embeddings=vector(item).astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [24]:
print(test[1].prompt)

How much does this cost to the nearest dollar?

Motorcraft YB3125 Fan Clutch
Motorcraft YB3125 Fan Clutch Package Dimensions 25.146 cms (L) x 20.066 cms (W) x 15.494 cms (H) Package Quantity 1 Product Type Auto Part Country Of Origin China Manufacturer Motorcraft, Brand Motorcraft, Model Fan Clutch, Weight 5 pounds, Dimensions 10 x 7.63 x 6.25 inches, Country of Origin China, model number Exterior Painted, Manufacturer Part Rank Automotive Automotive Replacement Engine Fan Clutches 583, Domestic Shipping can be shipped within U.S., International Shipping This item can be shipped to select countries outside of the U.S. Learn More, Available October 10, 2007

Price is $225.00


In [25]:
documents, prices = find_similars(test[1])

In [26]:
print(make_context(documents, prices))

To provide some context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Motorcraft Fan Clutch Assembly - YB3130
Motorcraft Fan Clutch Assembly - YB3130 Package Dimensions 7.23 L x 4 H x 6.92 W (inches) Package Weight 5 pounds Country of Origin China Part Number YB3130 Manufacturer Motorcraft, Brand Motorcraft, Model Fan Clutch Assembly, Weight 5 pounds, Dimensions 6.87 x 7.32 x 5.62 inches, Country of Origin China, model number Exterior Painted, Manufacturer Part Wattage 6.92 watts, Rank Automotive Automotive Replacement Engine Fan Clutches 698, Available October 10, 2007, Dimensions LxWxH 6.87 x 7.32 x 5.62 inches
Price is $223.78

Potentially related product:
Motorcraft YB3076 Fan Clutch
Motorcraft Fan Clutches are designed and tested to meet OE specifications for durability and reliability under extreme conditions. These clutches feature fluid which enhances the fan to turn faster and helps the engine reach operating t

In [56]:
messages_for(test[1], documents, prices)

'To provide some context, here are some other items that might be similar to the item you need to estimate.\n\nPotentially related product:\nMotorcraft Fan Clutch Assembly - YB3130\nMotorcraft Fan Clutch Assembly - YB3130 Package Dimensions 7.23 L x 4 H x 6.92 W (inches) Package Weight 5 pounds Country of Origin China Part Number YB3130 Manufacturer Motorcraft, Brand Motorcraft, Model Fan Clutch Assembly, Weight 5 pounds, Dimensions 6.87 x 7.32 x 5.62 inches, Country of Origin China, model number Exterior Painted, Manufacturer Part Wattage 6.92 watts, Rank Automotive Automotive Replacement Engine Fan Clutches 698, Available October 10, 2007, Dimensions LxWxH 6.87 x 7.32 x 5.62 inches\nPrice is $223.78\n\nPotentially related product:\nMotorcraft YB3076 Fan Clutch\nMotorcraft Fan Clutches are designed and tested to meet OE specifications for durability and reliability under extreme conditions. These clutches feature fluid which enhances the fan to turn faster and helps the engine reach o

In [28]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [29]:
get_price("The price for this is $99.99")

99.99

In [66]:
import time

def gemini_rag(item, delay=1.0):  # delay in seconds
    # Find similar items
    documents, prices = find_similars(item)
    
    response = handler.run(
        messages_for(item, documents, prices),max_retries=10000
    )
    # Clear conversation history
    handler.clear_history()    
    # 3. IMPORTANT: Still add the delay to avoid the IP rate limit
    print("--- Waiting for 2 seconds ---")
    time.sleep(2)
    
    # Return the processed price
    return get_price(response["text"])


In [53]:
# The function for gpt-4o-mini

def gpt_4o_mini_rag(item):
    documents, prices = find_similars(item)
    response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages_for(item, documents, prices),
        seed=42,
        max_tokens=5
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [67]:
gemini_rag(test[1])

➡️ Attempt 1/10000 using key from 'sijan.paudel10'...
Conversation history cleared.
--- Waiting for 2 seconds ---


224.78

In [36]:
test[1].price

225.11

In [ ]:
Tester.test(gemini_rag, test)

➡️ Attempt 1/10000 using key from 'paudelsijan15'...
Conversation history cleared.
--- Waiting for 2 seconds ---
1: Guess: $341.41 Truth: $374.41 Error: $33.00 SLE: 0.01 Item: OEM AC Compressor w/A/C Repair Kit For F...
➡️ Attempt 1/10000 using key from 'aayushkafle'...
Conversation history cleared.
--- Waiting for 2 seconds ---
2: Guess: $232.49 Truth: $225.11 Error: $7.38 SLE: 0.00 Item: Motorcraft YB3125 Fan Clutch
➡️ Attempt 1/10000 using key from 'aasutoshregmi'...
Conversation history cleared.
--- Waiting for 2 seconds ---
3: Guess: $63.99 Truth: $61.68 Error: $2.31 SLE: 0.00 Item: Dorman 603-159 Front Washer Fluid Reserv...
➡️ Attempt 1/10000 using key from 'pshreesha30'...
Conversation history cleared.
--- Waiting for 2 seconds ---
4: Guess: $365.00 Truth: $599.99 Error: $234.99 SLE: 0.25 Item: HP Premium 17.3-inch HD Plus Touchscreen...
➡️ Attempt 1/10000 using key from 'nepalcric4'...
Conversation history cleared.
--- Waiting for 2 seconds ---
5: Guess: $10.99 Truth: $16.99 E

In [ ]:
import time
from langchain_ollama import ChatOllama  # or ChatLlama if you use llama
from langchain_core.messages import HumanMessage, SystemMessage 

# Initialize your LLM (example using ChatOpenAI)
MODEL_LLAMA = 'llama3.2'
model = ChatOllama(model=MODEL_LLAMA)
def chat_rag(item, delay=2.0):  # delay in seconds
    # Find similar items
    documents, prices = find_similars(item)
    
    # Build the prompt/messages
    messages = [SystemMessage(content = system_message),
        HumanMessage(content=messages_for(item, documents, prices))
    ]
    
    # Call the model with retries if needed
    response = chat_model.invoke([messages], max_retries=10000)
    
    # Optional: clear any internal state if needed (depends on your handler)
    # If using a custom memory, you might call memory.clear() here
    
    # Delay to avoid rate limit
    print(f"--- Waiting for {delay} seconds to respect rate limits ---")
    time.sleep(delay)
    
    # Return processed price
    return get_price(response.generations[0][0].text)


In [ ]:
chat_rag

In [38]:
Tester.test(gemini_rag, test)

➡️ Attempt 1/11 using key from 'sijan.paudel10'...
Conversation history cleared.
1: Guess: $377.41 Truth: $374.41 Error: $3.00 SLE: 0.00 Item: OEM AC Compressor w/A/C Repair Kit For F...
➡️ Attempt 1/11 using key from 'paudelsijan15'...
Conversation history cleared.
2: Guess: $234.98 Truth: $225.11 Error: $9.87 SLE: 0.00 Item: Motorcraft YB3125 Fan Clutch
➡️ Attempt 1/11 using key from 'aayushkafle'...
Conversation history cleared.
3: Guess: $61.99 Truth: $61.68 Error: $0.31 SLE: 0.00 Item: Dorman 603-159 Front Washer Fluid Reserv...
➡️ Attempt 1/11 using key from 'aasutoshregmi'...
Conversation history cleared.
4: Guess: $365.00 Truth: $599.99 Error: $234.99 SLE: 0.25 Item: HP Premium 17.3-inch HD Plus Touchscreen...
➡️ Attempt 1/11 using key from 'pshreesha30'...
Conversation history cleared.
5: Guess: $13.99 Truth: $16.99 Error: $3.00 SLE: 0.03 Item: 5-Position Super Switch Pickup Selector ...
➡️ Attempt 1/11 using key from 'nepalcric4'...
Conversation history cleared.
6: Guess: $9.

RuntimeError: All API keys failed or quota exceeded after all attempts.

## Optional Extra: Trying a DeepSeek API call instead of OpenAI

If you have a DeepSeek API key, we will use it here as an alternative implementation; otherwise skip to the next section..

In [ ]:
# Connect to DeepSeek using the OpenAI client python library

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
deepseek_via_openai_client = OpenAI(api_key=deepseek_api_key,base_url="https://api.deepseek.com")

In [ ]:
# Added some retry logic here because DeepSeek is very oversubscribed and sometimes fails..

def deepseek_api_rag(item):
    documents, prices = find_similars(item)
    retries = 8
    done = False
    while not done and retries > 0:
        try:
            response = deepseek_via_openai_client.chat.completions.create(
                model="deepseek-chat", 
                messages=messages_for(item, documents, prices),
                seed=42,
                max_tokens=8
            )
            reply = response.choices[0].message.content
            done = True
        except Exception as e:
            print(f"Error: {e}")
            retries -= 1
    return get_price(reply)

In [ ]:
deepseek_api_rag(test[1])

In [ ]:
Tester.test(deepseek_api_rag, test)

## And now to wrap this in an "Agent" class

In [ ]:
from agents.frontier_agent import FrontierAgent

In [ ]:
# Let's print the logs so we can see what's going on

import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [ ]:
agent = FrontierAgent(collection)

In [ ]:
agent.price("Quadcast HyperX condenser mic for high quality podcasting")

In [ ]:
from agents.specialist_agent import SpecialistAgent

In [ ]:
agent2 = SpecialistAgent()

In [ ]:
agent2.price("Quadcast HyperX condenser mic for high quality podcasting")